In [ ]:
2. Data Analysis 
    
    A. How does the number of individual Robinhood accounts holding a given security affect the strike price of the underlying? How does this affect implied volatility?

    B. What is the correlation between price and volatility? For which stocks does the effect of Robinhood popularity have the greatest impact, and what insights do this offer on Robinhood's user base?

In [ ]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from MCForecastTools import MCSimulation

%matplotlib inline

In [4]:
from pathlib import Path
import pandas as pd

In [10]:
#### Functions 

# The total number of months included in the dataset 
def get_data_robintrack(filepath):
    df_robin_data = pd.read_csv(filepath)

    return df_robin_data


In [154]:
# Variables / Input Data
filepath = Path("../data/cleandata/Robintrack_Data/ABB_csv_clean.csv")

df_robin_data = get_data_robintrack(filepath)

# transform column 'timestamp' str into timestamp
df_robin_data['timestamp'] = pd.to_datetime(df_robin_data['timestamp'])

## resample groupying by day 
df_robin_data = df_robin_data.resample('D', on='timestamp').mean()

df_robin_data.head(25)


,users_holding
timestamp,
2020-01-01,2149.000000
2020-01-02,2145.958333
2020-01-03,2141.666667
2020-01-04,2147.000000
2020-01-05,2147.000000
2020-01-06,2147.000000
2020-01-07,NaN
2020-01-08,NaN
2020-01-09,NaN


In [157]:
import numpy as np
df_robin_data[np.isnan(df_robin_data)]=2147.00
df_robin_data.head(25)

,users_holding
timestamp,
2020-01-01,2149.000000
2020-01-02,2145.958333
2020-01-03,2141.666667
2020-01-04,2147.000000
2020-01-05,2147.000000
2020-01-06,2147.000000
2020-01-07,2147.000000
2020-01-08,2147.000000
2020-01-09,2147.000000


In [158]:
df_robin_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 223 entries, 2020-01-01 to 2020-08-10
Freq: D
Data columns (total 1 columns):
users_holding    223 non-null float64
dtypes: float64(1)
memory usage: 3.5 KB


In [ ]:
#### Functions 

# The total number of months included in the dataset 
def get_data_robintrack(filepath):
    with open(filepath, "r") as csvfile:            
        my_csvreader = csv.reader(csvfile)
        next(my_csvreader)                          #remove the column headers in cvs

        nr_months = len(list(my_csvreader))         #not including headers 
        str_nr_months = f"Total Months: {nr_months}"

    return str_nr_months

In [ ]:
# Variables / Input Data
filepath = Path("../data/Robintrack_Data/ABB.csv")
filepath_text = Path("../resources/results.txt")
list_of_functions = [f_nr_months, f_t_profit_losses, f_avg_change, f_greatest_increase, f_greatest_decrease]
list_of_results = []


for function in list_of_functions:
    result_func = function(filepath)
    print(result_func)
    list_of_results.append(result_func)

f_export_text_file (filepath_text, list_of_results)